## 생성한 번역모델 평가하기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets

In [ ]:
!pip install sacrebleu

In [ ]:
!pip install sentencepiece

In [5]:
# 데이터 셋을 다운받을 함수를 임포트 합니다.
from datasets import load_dataset

In [6]:
# 데이터을 다운로드 합니다.
dataset = load_dataset("Moo/korean-parallel-corpora")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [18]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ko', 'en'],
        num_rows: 96215
    })
    validation: Dataset({
        features: ['ko', 'en'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['ko', 'en'],
        num_rows: 2000
    })
})

In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [9]:
import torch

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [14]:
model_ckpt = "/content/drive/MyDrive/NLP 실습/1 25/ckpt/checkpoint-12000"

In [15]:
## 질문
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [27]:
from datasets import load_metric

sacrebleu = load_metric("sacrebleu")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [17]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [24]:
from tqdm import tqdm

# 모델을 사용하여 번역 수행
translated_texts = []

for example in tqdm(dataset['test']['en'][:100]):
    inputs = tokenizer.encode(example, return_tensors="pt", max_length=64, truncation=True)
    outputs = model.generate(inputs.to(device), max_length=64, num_beams=5, early_stopping=True)
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    translated_texts.append(translated_text)

100%|██████████| 100/100 [00:50<00:00,  1.98it/s]


In [36]:
dataset['test']['ko'][:5]

['토론에 참여한 사람들은 법 집행과 국가 안전보장에 대한 우려를 표명해야 할 필요성을 진지하게 받아 들이고 있습니다.',
 '또한 새로운 기술 개발이 어떤 해결책을 제공해 주는데 도움이 될 것이고, 동시 에 통신망 사용자들이 보다 안전하게 통신을 할 수 있도록 도움을 주게 되리라는 희망도 있습니다.',
 '그래서 클리퍼 칩에 대한 개인적인 해결책은 서서히 자취를 감출 것입니다. 그러나 이 문제는 계속 우리 곁에 존재할 것이며, 당분간은 심각한 상태로 존재할 것입니다.',
 '이탈리아의 천문학자들이 멀리에 있는 별들의 궤도를 도는 행성의 대기에서 생명체의 필수 요소인 물이 존재한다는 징후를 발견했다.',
 '물이 있다는 것이 다른 행성에 생명체들이 가득하다는 것을 의미하지는 않지만, 이번 발견이 확인된다면 그런 일이 가능할 수 있다는 추측을 증폭시키게 될 것이다.']

In [35]:
translated_texts[:5]

['이번 회담에 관여하는 사람들은 법에 대한 우려와 국가 안보에 대한 우려를 심각하게 생각하고 있다.',
 '이 같은 새로운 기술 개발은 일부 해결책을 제공하는데 도움이 될 것이며, 네트워크 사용자들이 통신에 대한 안전을 더 잘 유지할 수 있도록 도움을 줄 수 있을 것으로 기대하고 있다.',
 '그는 “그것은 보지 못할 것”이라며 “그것은 우리와 함께 해결될 것”이라고 말했다.',
 '이탈리아 천문학자들은 행성의 궤도에서 생존에 필요한 물질인 물의 징후를 발견했다.',
 '물이 있는 것은 다른 행성이 살아있을 수 있다는 것을 의미하는 것은 아니지만 이번 발견으로 다른 행성이 존재할 수 있다는 추측이 증폭될 것이다.']

In [31]:
refs = [[example] for example in dataset['test']['ko'][:100]]

In [32]:
bleu = sacrebleu.compute(predictions=translated_texts, references=refs)

In [33]:
bleu

{'score': 2.570544845929867,
 'counts': [377, 83, 21, 6],
 'totals': [1393, 1293, 1193, 1093],
 'precisions': [27.06389088298636,
  6.419180201082753,
  1.760268231349539,
  0.5489478499542544],
 'bp': 0.714134887023436,
 'sys_len': 1393,
 'ref_len': 1862}

In [37]:
meteor = load_metric('meteor')

/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for meteor contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/meteor/meteor.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [40]:
refs = [example for example in dataset['test']['ko'][:100]]  # 참조 번역

meteor_score = meteor.compute(predictions=translated_texts, references=refs)

In [41]:
meteor_score

{'meteor': 0.15935503008175408}

In [42]:
chrf = load_metric('chrf')

/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for chrf contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/chrf/chrf.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [44]:
chrf

Metric(name: "chr_f", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, usage: """
Produces ChrF(++) scores for hypotheses given reference translations.

Args:
    predictions (list of str): The predicted sentences.
    references (list of list of str): The references. There should be one reference sub-list for each prediction sentence.
    char_order (int): Character n-gram order. Defaults to `6`.
    word_order (int): Word n-gram order. If equals to `2`, the metric is referred to as chrF++. Defaults to `0`.
    beta (int): Determine the importance of recall w.r.t precision. Defaults to `2`.
    lowercase (bool): if `True`, enables case-insensitivity. Defaults to `False`.
    whitespace (bool): If `True`, include whitespaces when extracting character n-grams.
    eps_smoothing (bool): If `True`, applies epsilon smoothing similar
    to reference chrF++.py, NLTK and Moses im

In [45]:
refs = [[example] for example in dataset['test']['ko'][:100]]  # 참조 번역

chrf_score = chrf.compute(predictions=translated_texts, references=refs)

In [46]:
chrf_score

{'score': 17.88051124833359, 'char_order': 6, 'word_order': 0, 'beta': 2}

## XAI

In [63]:
from transformers import T5EncoderModel, AutoTokenizer

model_name = "/content/drive/MyDrive/NLP 실습/1 25/ckpt/checkpoint-12000"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5EncoderModel.from_pretrained(model_name).to(device) # 이때, T5모델의 인코더만 활용해서 분류할 수 있기 때문에 T5EncoderModel를 가져온다

In [48]:
sentences = ["The cat sat on the mat.", "It is raining.", "She loves reading books.", "This is a very long sentence with multiple words."]
labels = [1, 0, 1, 1]  # 5개 단어 이상은 1 (길음), 미만은 0 (짧음)

In [79]:
def extract_representations(model, tokenizer, sentences):
  model.eval()
  with torch.no_grad():
      representations = []
      for sentence in sentences:
          inputs = tokenizer(sentence, return_tensors="pt")
          outputs = model(**inputs.to(device))
          sentence_rep = outputs.last_hidden_state[:, 0, :]
          representations.append(sentence_rep.detach().cpu())

  return representations

In [80]:
representations = extract_representations(model, tokenizer, sentences)

In [81]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

In [82]:
X = torch.cat(representations).numpy()
y = np.array(labels)

clf = DecisionTreeClassifier()
clf.fit(X, y)

DecisionTreeClassifier()

In [83]:
test_sentences = ["A short sentence.", "This is another sentence.", "This is a much longer sentence with many more words than the others."]
test_labels = [0, 0, 1]

In [84]:
test_representations = extract_representations(model, tokenizer, test_sentences)
X_test = torch.cat(test_representations).numpy()
y_test = np.array(test_labels)

In [85]:
predictions = clf.predict(X_test)
accuracy = np.mean(predictions == y_test)

print(f"정확도: {accuracy}")

정확도: 0.3333333333333333


## 형태소 기반 평가

In [86]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

sentences = ["The cat is sleeping", "He reads a book", "They are running fast"]
labels = []

for sentence in sentences:
    words = word_tokenize(sentence)
    pos_tags = nltk.pos_tag(words)
    labels.append(any(tag.startswith('VB') for word, tag in pos_tags))

labels = [int(label) for label in labels]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [87]:
def extract_representations(model, tokenizer, sentences):
  model.eval()
  with torch.no_grad():
      representations = []
      for sentence in sentences:
          inputs = tokenizer(sentence, return_tensors="pt")
          outputs = model(**inputs.to(device))
          sentence_rep = outputs.last_hidden_state[:, 0, :]
          representations.append(sentence_rep.detach().cpu())

  return representations

representations = extract_representations(model, tokenizer, sentences)

In [88]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

X = torch.cat(representations).numpy()
y = np.array(labels)

clf = DecisionTreeClassifier()
clf.fit(X, y)

DecisionTreeClassifier()

In [89]:
test_sentences = ["She is singing", "The sun shines", "We went to the market"]
test_labels = []

for sentence in test_sentences:
    words = word_tokenize(sentence)
    pos_tags = nltk.pos_tag(words)
    test_labels.append(any(tag.startswith('VB') for word, tag in pos_tags))

test_labels = [int(label) for label in test_labels]

test_representations = extract_representations(model, tokenizer, test_sentences)
X_test = torch.cat(test_representations).numpy()
y_test = test_labels

predictions = clf.predict(X_test)
accuracy = np.mean(predictions == y_test)
print(f"정확도: {accuracy}")

정확도: 0.6666666666666666
